In [53]:
import numpy as np
import pandas as pd
from flask import Flask, request, jsonify
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import random
import os

In [ ]:
app = Flask(__name__)
# run_with_ngrok(app)

# Load fingerprint image paths dataset from CSV
fp_data = pd.read_csv('/content/fingerprints.csv')
fp_data.head(10)

# Load bank account details dataset from CSV
bank_data = pd.read_csv('/content/bank.csv')
bank_data.head(10)

In [56]:
# Read the existing data from bank.csv into a DataFrame
bank_data = pd.read_csv("/content/bank.csv")

# Generate random account numbers for the 'Account No' column
# Assuming account numbers are integers and you want unique account numbers
# Adjust the range and length of account numbers as needed
account_numbers = [random.randint(1000000000, 9999999999) for _ in range(len(bank_data))]

# Assign the generated random account numbers to the 'Account No' column
bank_data['Account No'] = account_numbers

# Write the modified data back to bank.csv
bank_data.to_csv("bank.csv", index=False)

bank_data.head()

,Account No,DATE,TRANSACTION DETAILS,CHQ.NO.,VALUE DATE,WITHDRAWAL AMT,DEPOSIT AMT,BALANCE AMT,.
0,5072611405,29-Jun-17,TRF FROM Indiaforensic SERVICES,NaN,29-Jun-17,NaN,"1,000,000.00","1,000,000.00",.
1,3358153089,5-Jul-17,TRF FROM Indiaforensic SERVICES,NaN,5-Jul-17,NaN,"1,000,000.00","2,000,000.00",.
2,2957783492,18-Jul-17,FDRL/INTERNAL FUND TRANSFE,NaN,18-Jul-17,NaN,"500,000.00","2,500,000.00",.
3,9265425334,1-Aug-17,TRF FRM Indiaforensic SERVICES,NaN,1-Aug-17,NaN,"3,000,000.00","5,500,000.00",.
4,6801022301,16-Aug-17,FDRL/INTERNAL FUND TRANSFE,NaN,16-Aug-17,NaN,"500,000.00","6,000,000.00",.


In [57]:
# Read the account numbers from bank.csv
bank_data = pd.read_csv("/content/bank.csv")

# Create a new column 'Account No' in fp_data and populate it with the account numbers
fp_data['Account No'] = bank_data['Account No']

# Write the modified data back to fingerprints.csv
fp_data.to_csv("fingerprints.csv", index=False)

fp_data.head()

,filename,width,height,pixel_values,Account No
0,364__M_Right_little_finger.BMP,96,103,[[[160 160 160 255]\n [160 160 160 255]\n [1...,5072611405
1,292__M_Left_little_finger.BMP,96,103,[[[160 160 160 255]\n [160 160 160 255]\n [1...,3358153089
2,53__M_Left_thumb_finger.BMP,241,298,[[[167 167 167]\n [151 151 151]\n [156 156 1...,2957783492
3,225__M_Left_little_finger.BMP,96,103,[[[160 160 160 255]\n [160 160 160 255]\n [1...,9265425334
4,277__M_Right_middle_finger.BMP,96,103,[[[160 160 160 255]\n [160 160 160 255]\n [1...,6801022301


In [58]:
fp_data.describe()
bank_data.describe()

,Account No,CHQ.NO.
count,1.162010e+05,905.000000
mean,5.512978e+09,791614.503867
std,2.598516e+09,151205.932910
min,1.000149e+09,1.000000
25%,3.267972e+09,704231.000000
50%,5.508297e+09,873812.000000
75%,7.777190e+09,874167.000000
max,9.999910e+09,874525.000000


In [59]:
print("Columns in fp_data:", fp_data.columns)
print("Columns in bank_data:", bank_data.columns)

Columns in fp_data: Index(['filename', 'width', 'height', 'pixel_values', 'Account No'], dtype='object')
Columns in bank_data: Index(['Account No', 'DATE', 'TRANSACTION DETAILS', 'CHQ.NO.', 'VALUE DATE',
       ' WITHDRAWAL AMT ', ' DEPOSIT AMT ', 'BALANCE AMT', '.'],
      dtype='object')


In [60]:
# Assuming you have a common key (In this case, Account No) to merge the datasets
merged_data = pd.merge(fp_data, bank_data, on='Account No')

merged_data.head()

,filename,width,height,pixel_values,Account No,DATE,TRANSACTION DETAILS,CHQ.NO.,VALUE DATE,WITHDRAWAL AMT,DEPOSIT AMT,BALANCE AMT,.
0,364__M_Right_little_finger.BMP,96,103,[[[160 160 160 255]\n [160 160 160 255]\n [1...,5072611405,29-Jun-17,TRF FROM Indiaforensic SERVICES,NaN,29-Jun-17,NaN,"1,000,000.00","1,000,000.00",.
1,292__M_Left_little_finger.BMP,96,103,[[[160 160 160 255]\n [160 160 160 255]\n [1...,3358153089,5-Jul-17,TRF FROM Indiaforensic SERVICES,NaN,5-Jul-17,NaN,"1,000,000.00","2,000,000.00",.
2,53__M_Left_thumb_finger.BMP,241,298,[[[167 167 167]\n [151 151 151]\n [156 156 1...,2957783492,18-Jul-17,FDRL/INTERNAL FUND TRANSFE,NaN,18-Jul-17,NaN,"500,000.00","2,500,000.00",.
3,225__M_Left_little_finger.BMP,96,103,[[[160 160 160 255]\n [160 160 160 255]\n [1...,9265425334,1-Aug-17,TRF FRM Indiaforensic SERVICES,NaN,1-Aug-17,NaN,"3,000,000.00","5,500,000.00",.
4,277__M_Right_middle_finger.BMP,96,103,[[[160 160 160 255]\n [160 160 160 255]\n [1...,6801022301,16-Aug-17,FDRL/INTERNAL FUND TRANSFE,NaN,16-Aug-17,NaN,"500,000.00","6,000,000.00",.


In [61]:
# Function to authenticate fingerprint
def authenticate_fingerprint(filename, account_no):
  if filename in merged_data['filename'].values:
        # Check if the account number matches
        if merged_data.loc[merged_data['filename'] == filename, 'Account No'].values[0] == account_no:
            return True  # Fingerprint authenticated
  return False

In [63]:
# Test the function
fingerprint_to_auth = '364__M_Right_little_finger.BMP'
account_no = 5072611405
is_auth = authenticate_fingerprint(fingerprint_to_auth, account_no)
print("Is fingerprint authenticated?", is_auth)

Is fingerprint authenticated? True


In [64]:
merged_data['authenticated'] = merged_data.apply(lambda row: authenticate_fingerprint(row['filename'], row['Account No']), axis=1)

In [65]:
# Feature Engineering
# Input features (fingerprint filename and account number)
X = merged_data[['filename', 'Account No']]

# Target labels (authentication status)
y = merged_data['authenticated']

In [66]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [79]:
# Reshape input features to be two-dimensional
X_train_reshaped = X_train.apply(lambda x: len(x)).values.reshape(-1, 1)
X_test_reshaped = X_test.apply(lambda x: len(x)).values.reshape(-1, 2)

print("Shape of X_test:", X_test.shape)
print("Shape of y_test:", y_test.shape)

print("Shape of X_test_reshaped:", X_test_reshaped.shape)

X_test_reshaped = X_test.values

Shape of X_test: (1200, 2)
Shape of y_test: (1200,)
Shape of X_test_reshaped: (1, 2)


In [80]:
# Check for duplicates in X_train_reshaped and y_train
duplicates_X_train = len(np.unique(X_train_reshaped)) != len(X_train_reshaped)
duplicates_y_train = len(np.unique(y_train)) != len(y_train)

print("Duplicates in X_train_reshaped:", duplicates_X_train)
print("Duplicates in y_train:", duplicates_y_train)

# Remove duplicates from X_train_reshaped
X_train_reshaped = np.unique(X_train_reshaped, axis=0)

# Remove duplicates from y_train
y_train = np.unique(y_train)

# If duplicates exist, remove them
if duplicates_X_train or duplicates_y_train:
    X_train_reshaped_unique, indices = np.unique(X_train_reshaped, axis=0, return_index=True)
    y_train_unique = y_train[indices]

    print("Shape after removing duplicates from X_train_reshaped:", X_train_reshaped_unique.shape)
    print("Shape after removing duplicates from y_train:", y_train_unique.shape)
else:
    print("No duplicates found.")

Duplicates in X_train_reshaped: True
Duplicates in y_train: False
Shape after removing duplicates from X_train_reshaped: (1, 1)
Shape after removing duplicates from y_train: (1,)


In [81]:
# Train a Random Forest classifier (for demonstration)
model = RandomForestClassifier(n_estimators=100, random_state=42)

# Fit the model
model.fit(X_train_reshaped, y_train)

RandomForestClassifier(random_state=42)

In [84]:
# Extract account numbers from X_test
X_test_account_numbers = X_test['Account No'].values.reshape(-1, 1)

# Predict labels for the test data
y_pred = model.predict(X_test_account_numbers)

# Calculate accuracy score
accuracy = accuracy_score(y_test, y_pred)

# Convert accuracy to percentage
accuracy_percent = accuracy * 100

print("Model Accuracy:", accuracy_percent, "%")

Model Accuracy: 100.0 %


In [50]:
# Define the API endpoint for fingerprint authentication
@app.route('/authenticate', methods=['POST'])
def authenticate_endpoints():
    data = request.json
    fingerprint_path = data.get('filename')
    account_number = data.get('Account No')
    if fingerprint_path and account_number:
        if os.path.exists(fingerprint_path):
            if authenticate_fingerprint(fingerprint_path, account_number):
                return jsonify({"authenticated": True})
    return jsonify({"authenticated": False})

In [22]:
if __name__ == '__main__':
    app.run(debug=True, host='0.0.0.0')